In [ ]:
# ========== LangChain 1.0 RAG 에이전트 실습 (문법 위주) ==========
# 필요한 패키지: pip install langchain langchain-text-splitters langchain-community langchain-openai langchain-core bs4

# --- 1) 모델 초기화 (문법: init_chat_model) ---
import os
from langchain.chat_models import init_chat_model

# 환경변수 OPENAI_API_KEY 또는 ANTHROPIC_API_KEY 필요
model = init_chat_model("gpt-4o-mini")  # OpenAI. 또는 "gpt-4.1"
# model = init_chat_model("claude-sonnet-4-5-20250929")  # Anthropic

## RAG 플로우 요약
1. **인덱싱**: 문서 로드 → 청크 분할 → 벡터 스토어 저장  
2. **검색·생성**: 리트리버 도구로 검색 → 에이전트가 도구 호출 후 답변 생성

In [ ]:
# --- 2) 임베딩 + 벡터 스토어 (문법) ---
# pip install langchain-openai
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 또는 text-embedding-3-large
vector_store = InMemoryVectorStore(embeddings)  # 메모리 저장소 (실무는 Chroma, FAISS 등)

In [ ]:
# --- 3) 문서 로드 (문법: DocumentLoader) ---
# langchain_community.document_loaders: WebBaseLoader, PyPDFLoader, TextLoader 등
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 웹 URL → Document 리스트. bs_kwargs로 파싱 영역 제한 가능
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),
)
docs = loader.load()  # list[Document], 각 Document는 page_content + metadata
print(f"로드된 문서 수: {len(docs)}, 첫 문서 길이: {len(docs[0].page_content)}")

In [ ]:
# --- 4) 문서 분할 (문법: TextSplitter) ---
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # 청크 최대 문자 수
    chunk_overlap=200,     # 청크 간 겹침
    add_start_index=True,  # 원본 내 위치 메타데이터
)
all_splits = text_splitter.split_documents(docs)  # list[Document]
print(f"총 {len(all_splits)}개 청크로 분할")

In [ ]:
# --- 5) 벡터 스토어에 저장 (인덱싱 완료) ---
# add_documents: 임베딩 후 저장, document_ids 반환
document_ids = vector_store.add_documents(documents=all_splits)
print("인덱싱 완료. ID 샘플:", document_ids[:3])

In [ ]:
# --- 6) RAG용 도구 정의 (문법: @tool) ---
# 에이전트가 "검색"을 할 때 이 함수가 호출됨
from langchain.tools import tool

@tool(response_format="content_and_artifact")  # 문자열 + 원본 문서를 artifact로 반환
def retrieve_context(query: str):
    """Retrieve information to help answer a query. 쿼리에 맞는 문서를 검색합니다."""
    retrieved_docs = vector_store.similarity_search(query, k=2)  # 상위 k개 유사 문서
    serialized = "\n\n".join(
        f"Source: {doc.metadata}\nContent: {doc.page_content}" for doc in retrieved_docs
    )
    return serialized, retrieved_docs  # (모델에 줄 텍스트, 앱에서 쓸 문서 리스트)

tools = [retrieve_context]

In [ ]:
# --- 7) 에이전트 생성 (문법: create_agent) ---
from langchain.agents import create_agent

# create_agent(model, tools, system_prompt=...)  ← 1.0 스타일
system_prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=system_prompt)

In [ ]:
# --- 8) 실행: invoke (한 번에 결과) ---
# 입력 형식: {"messages": [{"role": "user", "content": "질문"}]}
query = "What is task decomposition?"
result = agent.invoke({"messages": [{"role": "user", "content": query}]})
# result["messages"][-1] 이 최종 AI 응답
print(result["messages"][-1].content)

In [ ]:
# --- 9) 실행: stream (단계별 스트리밍) ---
# stream_mode="values": 매 단계마다 전체 state 스냅샷
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()  # 툴 호출 → 툴 결과 → 최종 답변 순으로 출력

### 문법 요약 (LangChain 1.0)
| 단계 | 문법 |
|------|------|
| 모델 | `init_chat_model("gpt-4o-mini")` from `langchain.chat_models` |
| 임베딩 | `OpenAIEmbeddings()` from `langchain_openai` |
| 벡터스토어 | `InMemoryVectorStore(embeddings)` from `langchain_core.vectorstores` |
| 로더 | `WebBaseLoader(web_paths=(url,)).load()` from `langchain_community.document_loaders` |
| 분할 | `RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)` |
| 저장 | `vector_store.add_documents(documents=all_splits)` |
| 도구 | `@tool(response_format="content_and_artifact")` def fn(query: str): ... from `langchain.tools` |
| 에이전트 | `create_agent(model, tools, system_prompt=...)` from `langchain.agents` |
| 실행 | `agent.invoke({"messages": [{"role":"user","content": query}]})` 또는 `agent.stream(..., stream_mode="values")` |